In [1]:
%pip install gym-super-mario-bros
%pip install -r requirements.txt 
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install 'stable-baselines3[extra]'

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import CheckpointCallback
from gym.wrappers import GrayScaleObservation
import os

In [3]:
from nes_py.wrappers import JoypadSpace
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from gym_super_mario_bros.actions import COMPLEX_MOVEMENT
env = gym.make('SuperMarioBros-v0')
env = JoypadSpace(env, COMPLEX_MOVEMENT)
# Grey scale
env = GrayScaleObservation(env, keep_dim=True)

/home/aayush_ad/Code/mario/.conda/lib/python3.8/site-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/home/aayush_ad/Code/mario/.conda/lib/python3.8/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/aayush_ad/Code/mario/.conda/lib/python3.8/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [4]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % (self.check_freq) == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [4]:
callback = CheckpointCallback(save_freq=100_000, save_path='./gym/')

In [5]:
done = True
env.reset()
for step in range(500):
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    # env.render()
    if done:
       state = env.reset()

/home/aayush_ad/Code/mario/.conda/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
/home/aayush_ad/Code/mario/.conda/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(done, (bool, np.bool8)):


In [4]:
import torch
torch.cuda.is_available()

True

In [7]:
!pip install optuna

In [12]:
def evaluate_model(model, env, num_episodes=5):
    """
    Evaluate the performance of the model on a given environment.

    Parameters:
    - model: The trained model to evaluate.
    - env: The environment on which to evaluate the model.
    - num_episodes: Number of episodes to run for the evaluation.

    Returns:
    - avg_reward: The average reward obtained over the evaluation episodes.
    """
    total_rewards = 0.0
    for ep in range(num_episodes):
        obs = env.reset()
        done = False
        rewards = 0.0
        for iter in range(1000):
            
            action = model.predict(obs)[0]
            obs, reward, done, info = env.step(int(action))
            rewards += reward
            if done:
                break
        total_rewards += rewards
        print(f"Episode {ep+1}: {rewards=:.2f}")
    avg_reward = total_rewards / num_episodes
    return avg_reward


In [ ]:
def wrap_env(env, stack):
    wrap_env = DummyVecEnv([lambda: env])
    wrap_env = VecFrameStack(env, stack, channels_order='last')
    return wrap_env

In [42]:
os.chdir('..')

In [43]:
import optuna

def objective(trial):
    lr = trial.suggest_uniform('lr', 1e-5, 8e-5)
    batch_size = trial.suggest_categorical('batch_size', [512, 1024, 2048])
    n_epochs = trial.suggest_int('n_epochs', 4, 10)
    clip_range = trial.suggest_uniform('clip_range', 0.26, 0.3)
    n_steps = trial.suggest_categorical('n_steps', [512, 1024])
    gamma = trial.suggest_loguniform('gamma', 0.99, 0.999)
    ent_coef = trial.suggest_loguniform('ent_coef', 0.00000001, 0.1)

    # Initialize and train model with suggested hyperparameters
    model = PPO("CnnPolicy", env, verbose=1, learning_rate=lr, n_steps=n_steps, batch_size=batch_size, n_epochs=n_epochs, clip_range=clip_range,ent_coef=ent_coef, gamma=gamma, tensorboard_log="logs", seed=43)
    model.learn(total_timesteps=100_000, tb_log_name="trial_{}".format(trial.number))
    
    # Evaluate the model
    rewards = evaluate_model(model, env)
    return rewards

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)


[I 2024-02-29 01:26:38,648] A new study created in memory with name: no-name-a8108947-2e85-48ed-bd32-f174076345c6
/tmp/ipykernel_125687/1788486069.py:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  lr = trial.suggest_uniform('lr', 1e-5, 8e-5)
/tmp/ipykernel_125687/1788486069.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  clip_range = trial.suggest_uniform('clip_range', 0.26, 0.3)
/tmp/ipykernel_125687/1788486069.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 0.99, 0.999)
/tmp/ipykernel_125687/17

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Logging to logs/trial_0_1
-----------------------------
| time/              |      |
|    fps             | 118  |
|    iterations      | 1    |
|    time_elapsed    | 8    |
|    total_timesteps | 1024 |
-----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 112           |
|    iterations           | 2             |
|    time_elapsed         | 18            |
|    total_timesteps      | 2048          |
| train/                  |               |
|    approx_kl            | 0.00037262624 |
|    clip_fraction        | 0             |
|    clip_range           | 0.269         |
|    entropy_loss         | -2.48         |
|    explained_variance   | 0.00131       |
|    learning_rate        | 3.66e-05      |
|    loss                 | 5.26          |
|    n_updat

[I 2024-02-29 01:48:34,622] Trial 0 finished with value: 986.2 and parameters: {'lr': 3.6551456738813514e-05, 'batch_size': 512, 'n_epochs': 4, 'clip_range': 0.2689188298105615, 'n_steps': 1024, 'gamma': 0.9944661006229739, 'ent_coef': 2.047411268820146e-08}. Best is trial 0 with value: 986.2.


Episode 5: rewards=1155.00
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Logging to logs/trial_1_1
-----------------------------
| time/              |      |
|    fps             | 75   |
|    iterations      | 1    |
|    time_elapsed    | 13   |
|    total_timesteps | 1024 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 69           |
|    iterations           | 2            |
|    time_elapsed         | 29           |
|    total_timesteps      | 2048         |
| train/                  |              |
|    approx_kl            | 0.0005045631 |
|    clip_fraction        | 0            |
|    clip_range           | 0.278        |
|    entropy_loss         | -2.48        |
|    explained_variance   | 0.00131      |
|    learning_rate        | 2.98e-05     |
|    loss                 | 6.77         |

[I 2024-02-29 02:20:50,662] Trial 1 finished with value: 552.0 and parameters: {'lr': 2.9785614909771627e-05, 'batch_size': 512, 'n_epochs': 5, 'clip_range': 0.27806016250444027, 'n_steps': 1024, 'gamma': 0.996265989726043, 'ent_coef': 0.037541053455221116}. Best is trial 0 with value: 986.2.


...

Episode 5: rewards=504.00
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Logging to logs/trial_8_1
-----------------------------
| time/              |      |
|    fps             | 50   |
|    iterations      | 1    |
|    time_elapsed    | 20   |
|    total_timesteps | 1024 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 23           |
|    iterations           | 2            |
|    time_elapsed         | 87           |
|    total_timesteps      | 2048         |
| train/                  |              |
|    approx_kl            | 0.0013693245 |
|    clip_fraction        | 0            |
|    clip_range           | 0.279        |
|    entropy_loss         | -2.48        |
|    explained_variance   | 0.00131      |
|    learning_rate        | 4.54e-05     |
|    loss                 | 5.23         |


[I 2024-02-29 08:17:25,307] Trial 8 finished with value: 1314.2 and parameters: {'lr': 4.539980921854614e-05, 'batch_size': 1024, 'n_epochs': 10, 'clip_range': 0.27929105913048885, 'n_steps': 1024, 'gamma': 0.9951330516816996, 'ent_coef': 0.002903213624740428}. Best is trial 8 with value: 1314.2.


Episode 5: rewards=1193.00
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Logging to logs/trial_9_1
----------------------------
| time/              |     |
|    fps             | 45  |
|    iterations      | 1   |
|    time_elapsed    | 11  |
|    total_timesteps | 512 |
----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 34           |
|    iterations           | 2            |
|    time_elapsed         | 29           |
|    total_timesteps      | 1024         |
| train/                  |              |
|    approx_kl            | 0.0012600031 |
|    clip_fraction        | 0            |
|    clip_range           | 0.287        |
|    entropy_loss         | -2.48        |
|    explained_variance   | -0.000685    |
|    learning_rate        | 5.13e-05     |
|    loss                 | 1.25         |
|    n

[I 2024-02-29 09:13:08,422] Trial 9 finished with value: 688.8 and parameters: {'lr': 5.134941776470676e-05, 'batch_size': 512, 'n_epochs': 9, 'clip_range': 0.28729233954979677, 'n_steps': 512, 'gamma': 0.9963062961373372, 'ent_coef': 7.279220744520372e-06}. Best is trial 8 with value: 1314.2.


Episode 5: rewards=504.00


In [44]:
best_params = study.best_params  # Get the best hyperparameters
print("Best hyperparameters:", best_params)

# Optionally, save best hyperparameters to a file
import json
with open("best_hyperparameters.json", "a") as f:
    json.dump(best_params, f)


Best hyperparameters: {'lr': 4.539980921854614e-05, 'batch_size': 1024, 'n_epochs': 10, 'clip_range': 0.27929105913048885, 'n_steps': 1024, 'gamma': 0.9951330516816996, 'ent_coef': 0.002903213624740428}


In [45]:
sorted_trials = sorted(study.trials, key=lambda trial: trial.value, reverse=study.direction == 'maximize')

with open("Trials3.txt", "a") as f:
    for trial in sorted_trials:
        f.write(str(trial))
        f.write("\n")


In [47]:
import pandas as pd

# Simulate extracting data from a study variable, assuming it's an Optuna study
# This is just an example; replace with actual study.trials iteration

# Placeholder for extracted trial data
trial_data = {
    "Trial Number": [],
    "State": [],
    "Values": [],
    "Datetime Start": [],
    "Datetime Complete": [],
    "Learning Rate (lr)": [],
    "Batch Size": [],
    "Epochs (n_epochs)": [],
    "Clip Range": [],
    "Steps (n_steps)": [],
    "Gamma": [],
    "ent_coef": []
}

# Example loop to simulate extracting data from study.trials (replace with actual data extraction)
for trial in study.trials:  # Assuming study is your Optuna study variable
    trial_data["Trial Number"].append(trial.number)
    trial_data["State"].append(trial.state)
    trial_data["Values"].append(trial.values)
    trial_data["Datetime Start"].append(trial.datetime_start)
    trial_data["Datetime Complete"].append(trial.datetime_complete)
    trial_data["Learning Rate (lr)"].append(trial.params["lr"])
    trial_data["Batch Size"].append(trial.params["batch_size"])
    trial_data["Epochs (n_epochs)"].append(trial.params["n_epochs"])
    trial_data["Clip Range"].append(trial.params["clip_range"])
    trial_data["Steps (n_steps)"].append(trial.params["n_steps"])
    trial_data["Gamma"].append(trial.params["gamma"])
    trial_data["ent_coef"].append(trial.params["ent_coef"])

# Convert to DataFrame
df_trials = pd.DataFrame(trial_data)

# Save to Excel
excel_path = "optuna_trials3.xlsx"
df_trials.to_excel(excel_path, index=False)

# Output the path for download or further use
print(f"Excel file saved at: {excel_path}")


Excel file saved at: optuna_trials3.xlsx


In [24]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 3.2 MB/s eta 0:00:0000:010:01


In [5]:
time = 2_000_000
lr = 0.0000453998092185461
batch_size = 1024
n_epochs = 10
clip_range = 0.279291059130489
n_steps = 1024
gamma = 0.9951330516817
ent_coef = 0.00290321362474043

seed = 43
callback = CheckpointCallback(save_freq=100_000, save_path='./gym/')
hyperparam = f'{time=}, {lr=}, {ent_coef=}, {batch_size=}, {n_epochs=}, {clip_range=}, {n_steps=}, {gamma=}'
model = PPO("CnnPolicy", env, verbose=1, learning_rate=lr, n_steps=n_steps, ent_coef=ent_coef, batch_size=batch_size, n_epochs=n_epochs, clip_range=clip_range, gamma=gamma, tensorboard_log="logs", seed=seed)
model.learn(total_timesteps=time, callback=callback, tb_log_name=hyperparam)

model.save("ppo_mario_{hyperparam}")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


/home/aayush_ad/Code/mario/.conda/lib/python3.8/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Logging to logs/time=2000000, lr=4.53998092185461e-05, ent_coef=0.00290321362474043, batch_size=1024, n_epochs=10, clip_range=0.279291059130489, n_steps=1024, gamma=0.9951330516817_2


/home/aayush_ad/Code/mario/.conda/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
/home/aayush_ad/Code/mario/.conda/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(done, (bool, np.bool8)):


-----------------------------
| time/              |      |
|    fps             | 130  |
|    iterations      | 1    |
|    time_elapsed    | 7    |
|    total_timesteps | 1024 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 75           |
|    iterations           | 2            |
|    time_elapsed         | 27           |
|    total_timesteps      | 2048         |
| train/                  |              |
|    approx_kl            | 0.0013693245 |
|    clip_fraction        | 0            |
|    clip_range           | 0.279        |
|    entropy_loss         | -2.48        |
|    explained_variance   | 0.00131      |
|    learning_rate        | 4.54e-05     |
|    loss                 | 5.23         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00419     |
|    value_loss           | 20.1         |
------------------------------------------
----------------

In [15]:
model=PPO.load('gym/rl_model_1100000_steps')

In [21]:
evaluate_model(model, env, num_episodes=5)

Episode 1: rewards=1242.00
Episode 2: rewards=1519.00
Episode 3: rewards=1420.00
Episode 4: rewards=1136.00
Episode 5: rewards=1118.00


1287.0

In [22]:
os.chdir('gym')

In [27]:
os.listdir()

['rl_model_100000_steps.zip',
 'best_model.zip',
 'rl_model_1400000_steps.zip',
 'rl_model_900000_steps.zip',
 'rl_model_1200000_steps.zip',
 'rl_model_500000_steps.zip',
 'rl_model_1500000_steps.zip',
 'rl_model_400000_steps.zip',
 'rl_model_700000_steps.zip',
 'rl_model_200000_steps.zip',
 'rl_model_1600000_steps.zip',
 'rl_model_1700000_steps.zip',
 'rl_model_1100000_steps.zip',
 'rl_model_1800000_steps.zip',
 'rl_model_800000_steps.zip',
 'rl_model_1000000_steps.zip',
 'rl_model_900000_steps (2).zip',
 'rl_model_1300000_steps.zip',
 'rl_model_600000_steps.zip',
 'rl_model_300000_steps.zip']

In [30]:
mp = {}

In [31]:
for file in os.listdir():
    if file.endswith('.zip') and file != 'rl_model_900000_steps (2).zip':
        model = PPO.load(file)
        print(file)
        # evaluate_model(model, env, num_episodes=5)
        mp[file] = evaluate_model(model, env, num_episodes=5)
       

rl_model_100000_steps.zip
Episode 1: rewards=755.00
Episode 2: rewards=632.00
Episode 3: rewards=755.00
Episode 4: rewards=504.00
Episode 5: rewards=632.00
best_model.zip
Episode 1: rewards=752.00
Episode 2: rewards=710.00
Episode 3: rewards=431.00
Episode 4: rewards=503.00
Episode 5: rewards=504.00
rl_model_1400000_steps.zip
Episode 1: rewards=1215.00
Episode 2: rewards=961.00
Episode 3: rewards=1408.00
Episode 4: rewards=1373.00
Episode 5: rewards=1022.00
rl_model_900000_steps.zip
Episode 1: rewards=1367.00
Episode 2: rewards=1275.00
Episode 3: rewards=1445.00
Episode 4: rewards=1368.00
Episode 5: rewards=1022.00
rl_model_1200000_steps.zip
Episode 1: rewards=1048.00
Episode 2: rewards=1064.00
Episode 3: rewards=1147.00
Episode 4: rewards=1003.00
Episode 5: rewards=1365.00
rl_model_500000_steps.zip
Episode 1: rewards=1422.00
Episode 2: rewards=961.00
Episode 3: rewards=1298.00
Episode 4: rewards=1026.00
Episode 5: rewards=1387.00
rl_model_1500000_steps.zip
Episode 1: rewards=1290.00
E

In [32]:

mp

{'rl_model_100000_steps.zip': 655.6,
 'best_model.zip': 580.0,
 'rl_model_1400000_steps.zip': 1195.8,
 'rl_model_900000_steps.zip': 1295.4,
 'rl_model_1200000_steps.zip': 1125.4,
 'rl_model_500000_steps.zip': 1218.8,
 'rl_model_1500000_steps.zip': 1212.8,
 'rl_model_400000_steps.zip': 680.0,
 'rl_model_700000_steps.zip': 1131.8,
 'rl_model_200000_steps.zip': 711.4,
 'rl_model_1600000_steps.zip': 852.2,
 'rl_model_1700000_steps.zip': 1115.0,
 'rl_model_1100000_steps.zip': 1266.4,
 'rl_model_1800000_steps.zip': 1145.0,
 'rl_model_800000_steps.zip': 1080.8,
 'rl_model_1000000_steps.zip': 886.8,
 'rl_model_1300000_steps.zip': 876.4,
 'rl_model_600000_steps.zip': 1188.6,
 'rl_model_300000_steps.zip': 902.0}

In [33]:
from stable_baselines3.common.evaluation import evaluate_policy

In [36]:
model = PPO.load('rl_model_900000_steps')

In [40]:
evaluate_model(model, env, num_episodes=5)

Episode 1: rewards=1367.00
Episode 2: rewards=1275.00
Episode 3: rewards=1445.00
Episode 4: rewards=1368.00
Episode 5: rewards=1022.00


1295.4